In [146]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint
from copy import deepcopy
import dill
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 27)

In [2]:
## initial data needed to generate setup of rotors through Bombe Scrambler
# entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
# forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}
# rev_rotors = {}
# for r in raw_rotors.keys():
#     working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
#     rev_rotors[r] = [working[k] for k in sorted(working.keys())]
# reflectors = {'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 
#               'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}

entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
notches = {'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}
## forward rotors is the forward in:out pairings of each rotor as the character index of the A-Z ascii alphabet stored in 'entry'
forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}

## couldn't see a straightforward list comprehension for the reverse - the in:out pairing for when the current flows
## back from the reflector to the final output. Hopefully this two-step for loop isn't too slow
rev_rotors = {}
for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

# for k,v in rev_rotors.items():
#     print(k,':',v)

## in:out pairings for reflectors are hardcoded in, were originally generated by code
reflectors = {'B': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'C': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}
# print(reflectors)
blank_status = {char:0 for char in entry}

In [265]:
# %%timeit
# x = deepcopy(blank_status)
# x = {char:0 for char in entry}
# print(x)

In [266]:
class ScramblerBombe:
    
    def __init__(self,left_rotor,middle_rotor,right_rotor,reflector,menu_link='ZZZ',conx_in={},conx_out={}):
        """rotors must be strings referring to either ['I','II','III','IV','V']
        reflector must be string, one of either ['B','C']"""
        
        self.right_rotor = right_rotor
        self.middle_rotor = middle_rotor
        self.left_rotor = left_rotor
        self.reflector = reflectors[reflector]
        self.menu_link = menu_link
        self.middle_notch = entry.index(notches[self.middle_rotor])   ## point if right rotor reaches will trigger middle rotor to step
        self.left_notch = entry.index(notches[self.left_rotor])  ## point if middle rotor reaches will trigger left rotor to step
        self.current_position = menu_link
        self.pos_left_rotor, self.pos_mid_rotor, self.pos_rgt_rotor = (ascii_uppercase.index(m) for m in menu_link.upper())
        self.status = {}
        self.status['in'] = {char:0 for char in entry}
        self.status['out'] = {char:0 for char in entry}
        self.conx_in = conx_in
        self.conx_out = conx_out
        
    def once_thru_scramble(self,start_character, direction, first_rotor, pos1, second_rotor, pos2, 
                       third_rotor, pos3):
        """ start_character must be single ASCII character A-Z
        direction is either 'forward' or 'back' """
        if direction == 'forward':
            usedict = {k:v for k,v in forward_rotors.items()}
        elif direction == 'back':
            usedict = {k:v for k,v in rev_rotors.items()}
        else:
            print('only forward or back for direction')
            return 'wtf'

        start_character = start_character.upper()
        entry_pos = entry.index(start_character)
        fst_pos_modifier = (26 + pos1 - 0)%26
        fst_in = (entry_pos + fst_pos_modifier)%26
        fst_out = usedict[first_rotor][fst_in]
        ch1o = entry[fst_out]

        scd_pos_modifier = (26 + pos2 - pos1)%26
        scd_in = (fst_out + scd_pos_modifier)%26
        ch2i = entry[scd_in]
        scd_out = usedict[second_rotor][scd_in]
        ch2o = entry[scd_out]

        thd_pos_modifier = (26 + pos3 - pos2)%26
        thd_in = (scd_out + thd_pos_modifier)%26
        ch3i = entry[thd_in]
        thd_out = usedict[third_rotor][thd_in]
        ch3o = entry[thd_out]

#         if direction == 'forward':
#             print(f"{start_character} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
#         elif direction == 'back':
#             print(f"{start_character} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")

        return ch3o
    
    def full_scramble(self,in_ch):
        in_ch = in_ch.upper()
        left_rotor = self.left_rotor
        middle_rotor = self.middle_rotor
        right_rotor = self.right_rotor
        rflector = self.reflector
        ## first run right to left through scrambler
        forward_run = self.once_thru_scramble(in_ch, direction='forward', first_rotor=right_rotor, pos1=self.pos_rgt_rotor, 
                                              second_rotor=middle_rotor, pos2=self.pos_mid_rotor, 
                                              third_rotor=left_rotor, pos3=self.pos_left_rotor)

        ## reflector back around for return
        rfi_pos_mod = (26 + 0 - self.pos_left_rotor)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
        rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
        chri = entry[rf_in]
        mirrored = rflector[chri]

#         print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")

        ## second run back left to right thru scrambler
        back_run = self.once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=self.pos_left_rotor, 
                                      second_rotor=middle_rotor, pos2=self.pos_mid_rotor, third_rotor=right_rotor, pos3=self.pos_rgt_rotor)

        bk_out = entry.index(back_run)
        bko_pos_mod = (26 + 0 - self.pos_rgt_rotor)%26  ## as above, '0' just reflects that the entry interface doesn't move
        bk_final = (bk_out + bko_pos_mod)%26
        final = entry[bk_final]
#         print('RR back out:  ', back_run, '-->', final)
#         print(in_ch,"-->",final)
        return final
    
    def rotor_step(self,rotor_position):
        """"""
        if rotor_position == 25:
            rotor_position = 0
        else:
            rotor_position += 1
        return rotor_position
    
    def translate_current_position(self):
        self.current_position = ''
        for pos in self.pos_left_rotor,self.pos_mid_rotor,self.pos_rgt_rotor:
            self.current_position += entry[pos]
    
    def step_enigma(self):
        """Just acts on itself, steps the rotors"""
#         print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")
        if self.pos_rgt_rotor == self.middle_notch and self.pos_mid_rotor == self.left_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            self.pos_left_rotor = self.rotor_step(self.pos_left_rotor)
#             print('--- left & middle rotor step')
        elif self.pos_rgt_rotor == self.middle_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
#             print('--- middle rotor step ---')
        else:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
        
        self.translate_current_position()           
#         print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")

    def update(self):
        """idea here is that the scrambler will check each of the 26 connections to see if they
        are live, and if so pass it through itself to live up the connection on the other side of 
        the scrambler"""
        for sides in [['in','out'],['out','in']]:
            for char, io in self.status[sides[0]].items():
                if io == 0:
                    pass
                else:
                    self.status[sides[1]][self.full_scramble(char)] = 1
                    
    def reset_status(self):
        self.status['in'] = {char:0 for char in entry}
        self.status['out'] = {char:0 for char in entry}
        



In [4]:
# eb1 = ScramblerBombe(left_rotor='II',middle_rotor='V',right_rotor='III',reflector='C',menu_link='BCD')

In [5]:
# eb1.translate_current_position()
# eb1.current_position

In [6]:
# eb1.status['in']['E'] = 1
# eb1.status['in']['H'] = 1
# eb1.status['out']['P'] = 1
# eb1.status['out']['X'] = 1
# print(eb1.status)

In [7]:
# eb1.update()
# print(eb1.status)

In [8]:
# %%time
# for i in range(26**3):
#     eb1.full_scramble('Q')
#     eb1.step_enigma()

In [9]:
# word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [10]:
# def only_ascii(instring):
#     """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
#     newstring = ''
#     for character in instring:
#         if character in ascii_letters:
#             newstring += character.upper()
#     return newstring

In [11]:
# only_ascii(word)

In [267]:
class Bombe:
    
    def __init__(self,left_rotor='I',middle_rotor='II',right_rotor='III',reflector='B'):
        self.left_rotor = left_rotor
        self.middle_rotor = middle_rotor
        self.right_rotor = right_rotor
        self.reflector = reflector
        self.scramblers = {}
        self.register = {'status':{char:0 for char in entry}}
        self.current_sum = sum(self.register['status'].values())
        self.run_record = {}
        
    def setup(self,menu):
        self.menu = menu
        for key,val in self.menu.items():
            if key == 'config':
                self.test_char = self.menu['config']['test_char']
                self.register['conxns'] = self.menu['config']['conx_in']
            else:
                self.scramblers[key] = ScramblerBombe(self.left_rotor,self.middle_rotor,self.right_rotor,self.reflector,
                                                      menu_link=val['menu_link'],conx_in=val['conx_in'], conx_out=val['conx_out'])
        self.lowest_scrambler_order = min([key for key in self.scramblers.keys()])
        self.identity_scrambler = self.scramblers[self.lowest_scrambler_order]
    
    def pulse_connections(self):
        """Goes through every scrambler (once only), and updates in and out terminals with live feeds via conx_in and
        conx_out from its connected scramblers. Should be run iteratively to exhaustion."""
        for order,scr in self.scramblers.items():
            ## i.e for each scrambler in the enigma, we're going to loop through it's conx_in and conx_out dict
            ## and check its connected scramblers to match any live wires
            for pos,side in scr.conx_in.items():
                ## for this scrambler, go through the conx_in dict. It has the other scrambler you need to 
                ## look at (pos, or the number/position label) and whether you're looking at the 'in' or 'out' side
                for ch,io in self.scramblers[pos].status[side].items():
                    # look through the scrambler's status for that side (in or out)
                    if io == 1:   # and if it has a live wire
                        scr.status[side][ch] = 1 # set the corresponding character in the conx_in dict of the 
                                                  # original scrambler being checked to also be live

            for pos,side in scr.conx_out.items():   ## repeating as above for conx_out
                for ch,io in self.scramblers[pos].status[side].items():
                    if io == 1:
                        scr.status[side][ch] = 1

                
    def sync_test_register(self):
        
        for pos,side in self.register['conxns'].items():
            ## for each of the scrambler terminals connected to the test register
            for ch,io in self.scramblers[pos].status[side].items():
                ## first sync any live wires from the scrambler to the test register
                if io == 1:   
                    self.register['status'][ch] = 1
            for ch,io in self.register['status'].items():
                ## but then also sync any live wires from the test register to the scrambler
                if io == 1:
                    self.scramblers[pos].status[side][ch] = 1           

    
    def light_character(self,in_character):
        """just need a way to put in the initial character input into all of the scramblers"""
        self.register['status'][self.test_char] = 1
    
    def update_all(self):
        """For every scrambler, runs update() which passes live terminals through the scrambler - from in to out
        and vice versa, for whatever the current position is"""
        for scr in self.scramblers.keys():
            self.scramblers[scr].update()
            
    def spin_scramblers(self):
        """Runs step_enigma for all scramblers, spinning the right rotor once and perhaps the middle and left if
        they are in their notch positions"""
        for scr in self.scramblers.values():
            scr.step_enigma()
            
    def reset_scramblers_and_register(self):
        for scr in self.scramblers.values():
            scr.reset_status()
        self.register['status'] = {char:0 for char in entry}
    
    def check_this_lineup(self):
        """For running to exhaustion on a particular bombe scrambler lineup. 
        Loops through pulsing connections between scramblers and syncing back to the test register
        until the sum of live connections at the test register remains unchanged for two successive loops."""
        self.reset_scramblers_and_register()   # first make sure all scrambler inputs/outputs (statuses) are reset to zero
        self.light_character(self.test_char)   # light up the one test character
        self.sync_test_register()              # do the first syncing of test register, sending the signal out to the 
                                                # scramblers which are connected to the test register
            
    ### initialise the three sum variables to keep track of whether the sum of live connections have remained unchanged
#         print(self.register['status'])
        self.current_sum = sum(self.register['status'].values())
#         print(self.current_sum)
        olderer_sum = 0
        previous_sum = 0
        self.lineup_iters = 0   # this is just to keep track of how many iterations it took to reach a steady status
        while not (self.current_sum == previous_sum == olderer_sum):
            print(f"BEFORE  olderer:{olderer_sum}    prev: {previous_sum}     curr: {self.current_sum}")
            olderer_sum = previous_sum
            previous_sum = self.current_sum
            self.update_all()
            self.pulse_connections()
            self.sync_test_register()
            print([k for k,v in self.register['status'].items() if v == 1])
            self.current_sum = sum(self.register['status'].values())
            print(f"AFTER   olderer:{olderer_sum}    prev: {previous_sum}     curr: {self.current_sum}")
            self.lineup_iters += 1
    
    def step_and_test(self):
        self.spin_scramblers()
        self.check_this_lineup()
        self.run_record[self.identity_scrambler.current_position] = (self.current_sum,self.lineup_iters)
        if self.current_sum != 26:
            print('drop')
            print(self.identity_scrambler.current_position)
            

    def pdf_scrambler_statuses(self):
        bigdict = {}
        bigdict['REG'] = {k:('X' if v==1 else '') for k,v in self.register['status'].items()}
        for pos,scr in self.scramblers.items():
            littledict = {k:{l:(k[0].upper() if r == 1 else '') for l,r in v.items()} for k,v in scr.status.items()}
            bigdict[pos] ={k1:v1+littledict['out'][k1] for k1,v1 in littledict['in'].items()}
        self.flash_scrambler_statuses = pd.DataFrame(bigdict).T

In [280]:
with open('menu_transfer.pickle', 'rb') as infile:
    menu = dill.load(infile)
# menu

In [293]:
tb1 = Bombe('I','II','III','B')
tb1.setup(menu)
try:
    del(scr)
except:
    pass

In [294]:
# %%time
while tb1.identity_scrambler.current_position != 'IOA':
    tb1.spin_scramblers()

In [295]:
tb1.test_char = 'C'

In [296]:
tb1.spin_scramblers()
tb1.reset_scramblers_and_register()
print(tb1.identity_scrambler.current_position)
tb1.light_character(tb1.test_char)
tb1.sync_test_register()

IOB


<font color = 'red'><font size = 5>
Looks like there is a problem with the identity scrambler not lighting up?

In [304]:
tb1.update_all()
tb1.pulse_connections()
tb1.sync_test_register()
tb1.pdf_scrambler_statuses()
tb1.flash_scrambler_statuses

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
REG,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X
4,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
13,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
16,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
6,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
2,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
14,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
10,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
11,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO
9,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO,IO


In [327]:
# tb1.identity_scrambler.current_position

In [328]:
# new_record = {}
# for pos,scr in tb1.scramblers.items():
# #     print(menu[pos]['in'])
#     zig = menu[pos]['in']
#     should = menu[pos]['out']
#     zag = scr.full_scramble(zig)
#     spacer=''
#     if pos < 10:
#         spacer = ' '
#     print(pos,spacer,zig,'-->',zag,'/ sh:',should,zag == should)
#     new_record[pos] = {'in':zig,'out':zag,'current_pos':scr.current_position,'should':should}

In [314]:
# tb1.step_and_test()

In [310]:
# tb1.identity_scrambler.status

In [232]:
%%time
for i in range(1000):#26**3):
    tb1.step_and_test()

In [233]:
{k:v for k,v in tb1.run_record.items() if v[0] != 26}

In [27]:
# tb1.run_record

In [73]:
# for order,scr in tb1.scramblers.items():
#     print(order, [k for k,v in scr.status['in'].items() if v == 1], [k for k,v in scr.status['out'].items() if v == 1])

In [198]:
tword = 'OHWHATABEAUTIFULMORNINGOH'

In [203]:
eb2 = ScramblerBombe('I','II','III','B','IND')

In [313]:

result = ''
for i,t in enumerate(tword):
    eb2.step_enigma()
    r = eb2.full_scramble(t)
    print(i,eb2.current_position, t, '-->', r)
    result += r


In [205]:
result

'NKNMWGEDRLLLTROMZYLWPHVZJ'

In [284]:
original_record = {23: {'in': 'O', 'out': 'Z', 'current_pos': 'IOB'},
 24: {'in': 'H', 'out': 'J', 'current_pos': 'IOC'},
 25: {'in': 'W', 'out': 'Q', 'current_pos': 'IOD'},
 26: {'in': 'H', 'out': 'G', 'current_pos': 'IOE'},
 27: {'in': 'A', 'out': 'F', 'current_pos': 'IPF'},
 28: {'in': 'T', 'out': 'R', 'current_pos': 'IPG'},
 29: {'in': 'A', 'out': 'C', 'current_pos': 'IPH'},
 30: {'in': 'B', 'out': 'T', 'current_pos': 'IPI'},
 31: {'in': 'E', 'out': 'T', 'current_pos': 'IPJ'},
 32: {'in': 'A', 'out': 'T', 'current_pos': 'IPK'},
 33: {'in': 'U', 'out': 'J', 'current_pos': 'IPL'},
 34: {'in': 'T', 'out': 'J', 'current_pos': 'IPM'},
 35: {'in': 'I', 'out': 'A', 'current_pos': 'IPN'},
 36: {'in': 'F', 'out': 'D', 'current_pos': 'IPO'},
 37: {'in': 'U', 'out': 'Q', 'current_pos': 'IPP'},
 38: {'in': 'L', 'out': 'X', 'current_pos': 'IPQ'},
 39: {'in': 'D', 'out': 'C', 'current_pos': 'IPR'},
 40: {'in': 'A', 'out': 'O', 'current_pos': 'IPS'},
 41: {'in': 'Y', 'out': 'V', 'current_pos': 'IPT'}}

In [292]:
new_record

{4: {'in': 'A', 'out': 'F', 'current_pos': 'IPF', 'should': 'F'},
 13: {'in': 'F', 'out': 'K', 'current_pos': 'IOO', 'should': 'D'},
 16: {'in': 'D', 'out': 'Z', 'current_pos': 'IOR', 'should': 'C'},
 6: {'in': 'C', 'out': 'X', 'current_pos': 'IOH', 'should': 'A'},
 2: {'in': 'Q', 'out': 'W', 'current_pos': 'IOD', 'should': 'W'},
 14: {'in': 'U', 'out': 'A', 'current_pos': 'IOP', 'should': 'Q'},
 10: {'in': 'J', 'out': 'R', 'current_pos': 'IOL', 'should': 'U'},
 11: {'in': 'T', 'out': 'G', 'current_pos': 'IOM', 'should': 'J'},
 9: {'in': 'T', 'out': 'B', 'current_pos': 'IOK', 'should': 'A'},
 5: {'in': 'T', 'out': 'R', 'current_pos': 'IPG', 'should': 'R'},
 17: {'in': 'A', 'out': 'K', 'current_pos': 'IOS', 'should': 'O'},
 0: {'in': 'O', 'out': 'Z', 'current_pos': 'IOB', 'should': 'Z'}}

In [295]:
for pos in sorted(new_record.keys()):
    dval = new_record[pos]
    odict = original_record[pos+23]
    npos = dval['current_pos']
    opos = odict['current_pos']
    print(pos,dval,odict)

0 {'in': 'O', 'out': 'Z', 'current_pos': 'IOB', 'should': 'Z'} {'in': 'O', 'out': 'Z', 'current_pos': 'IOB'}
2 {'in': 'Q', 'out': 'W', 'current_pos': 'IOD', 'should': 'W'} {'in': 'W', 'out': 'Q', 'current_pos': 'IOD'}
4 {'in': 'A', 'out': 'F', 'current_pos': 'IPF', 'should': 'F'} {'in': 'A', 'out': 'F', 'current_pos': 'IPF'}
5 {'in': 'T', 'out': 'R', 'current_pos': 'IPG', 'should': 'R'} {'in': 'T', 'out': 'R', 'current_pos': 'IPG'}
6 {'in': 'C', 'out': 'X', 'current_pos': 'IOH', 'should': 'A'} {'in': 'A', 'out': 'C', 'current_pos': 'IPH'}
9 {'in': 'T', 'out': 'B', 'current_pos': 'IOK', 'should': 'A'} {'in': 'A', 'out': 'T', 'current_pos': 'IPK'}
10 {'in': 'J', 'out': 'R', 'current_pos': 'IOL', 'should': 'U'} {'in': 'U', 'out': 'J', 'current_pos': 'IPL'}
11 {'in': 'T', 'out': 'G', 'current_pos': 'IOM', 'should': 'J'} {'in': 'T', 'out': 'J', 'current_pos': 'IPM'}
13 {'in': 'F', 'out': 'K', 'current_pos': 'IOO', 'should': 'D'} {'in': 'F', 'out': 'D', 'current_pos': 'IPO'}
14 {'in': 'U', '

In [297]:
for scr in tb1.scramblers.values():
    print(scr.left_notch, scr.middle_notch)

16 4
16 4
16 4
16 4
16 4
16 4
16 4
16 4
16 4
16 4
16 4
16 4


In [317]:
for i in range(10):
    tb1.identity_scrambler.step_enigma()
    print(tb1.identity_scrambler.current_position)

Lpos=8, Mpos=14, Rpos=1
IOB
Lpos=8, Mpos=14, Rpos=2
IOC
Lpos=8, Mpos=14, Rpos=3
IOD
Lpos=8, Mpos=14, Rpos=4
IOE
Lpos=8, Mpos=15, Rpos=5
IPF
Lpos=8, Mpos=15, Rpos=6
IPG
Lpos=8, Mpos=15, Rpos=7
IPH
Lpos=8, Mpos=15, Rpos=8
IPI
Lpos=8, Mpos=15, Rpos=9
IPJ
Lpos=8, Mpos=15, Rpos=10
IPK
